# **Important Installations & Dependencies**


In [2]:
import re
import string
import math
import numpy as np
import pandas as pd
import csv
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
import operator
from math import log
!pip install farasapy
from farasa.stemmer import FarasaStemmer
from sklearn.model_selection  import train_test_split
from sklearn.utils import shuffle
import itertools
import collections
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import BernoulliNB
from sklearn import metrics
from sklearn.metrics import f1_score, precision_score,recall_score
from sklearn.metrics import classification_report as creport
from multiprocessing import Pool

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **Load the Binary Datset**

In [ ]:
from google.colab import files
uploaded= files.upload ()

Saving BinaryPublicFigures.xlsx to BinaryPublicFigures.xlsx


In [ ]:
import io
df= pd.read_excel(io.BytesIO(uploaded['BinaryPublicFigures.xlsx'])) 
df= df[['Text','Label']]
df
# path='/content/BinaryPublicFigures.xlsx'
# df = pd.read_excel(path)
# df= df['Text','Label']


### **Load the Multi Datset**

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Art= pd.read_excel(io.BytesIO(uploaded['Art.xlsx']))
# Art= Art[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Business= pd.read_excel(io.BytesIO(uploaded['Business.xlsx']))
# Business= Business[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Politics= pd.read_excel(io.BytesIO(uploaded['Politics.xlsx']))
# Politics= Politics[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Judiciary= pd.read_excel(io.BytesIO(uploaded['Judiciary.xlsx']))
# Judiciary= Judiciary[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Sports= pd.read_excel(io.BytesIO(uploaded['Sports.xlsx']))
# Sports= Sports[['Text','Label']]

In [ ]:
# from google.colab import files
# uploaded= files.upload ()

In [ ]:
# import io
# Literature= pd.read_excel(io.BytesIO(uploaded['Literature.xlsx']))
# Literature= Literature[['Text','Label']]

In [ ]:
# frames= [Literature,Sport, Judiciary, Politics, Art, Business]
# df= pd.concat(frames)

# **Data Preprocessing**


In [ ]:
def clean(df):
    df = remove_diacritics(df)
    df = normalize_arabic(df)
    df = remove_punctuations(df)
    df = remove_repeating_char(df)
    df= remove_english_word_and_numbers(df)
    df=clean_space(df)
    return df

In [ ]:
arabic_punctuations = '''`÷« »×؛<>٩٨'٧٦٥٤٣٢١٠_()↗*•&^%][ـ،/:"؟.,'{}⋮≈~¦+|٪!”…“–ـ/[]%=#*+\\•~@£·_{}©^®`→°€™›♥←×§″′Â█à…“★”–●â►−¢¬░¶↑±▾	═¦║―¥▓—‹─▒：⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡₹´'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


In [ ]:
def remove_diacritics(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_diacritics(x))
    return df
def _remove_diacritics(x):
    x = str(x)
    x = re.sub(arabic_diacritics, '', x)
    return x

def normalize_arabic(df):
    df['Text'] = df['Text'].apply(lambda x: _normalize_arabic(x))
    return df
def _normalize_arabic(x):
    x = str(x)
    # added space around puncts after replace
    x = re.sub("[إأآا]", "ا", x)
    x = re.sub("ى", "ي", x)
    x = re.sub("ؤ", "ء", x)
    x = re.sub("ئ", "ء", x)
    x = re.sub("ة", "ه", x)
    x = re.sub("گ", "ك", x)
    return x

def remove_punctuations(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_punctuations(x))
    return df
def _remove_punctuations(x):
    x = str(x)
    #translator = str.maketrans(' ', ' ', punctuations_list)
    translator = str.maketrans(punctuations_list, ' '*len(punctuations_list))
    return x.translate(translator)

def remove_repeating_char(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_repeating_char(x))
    return df
def _remove_repeating_char(x):
    x = str(x)
    return re.sub(r'(.)\1+', r'\1', x)

def remove_english_word_and_numbers(df):
    df['Text'] = df['Text'].apply(lambda x: _remove_english_word_and_numbers(x))
    return df
def _remove_english_word_and_numbers(x):
    x = str(x)
    return re.sub(r'[a-zA-Z0-9]+', '', x)

def clean_space(df):
    compiled_re = re.compile(r"\s+")
    df['Text'] = df["Text"].apply(lambda x: _clean_space(x, compiled_re))
    return df
def _clean_space(x, compiled_re):
    return compiled_re.sub(" ", x)


In [ ]:
text_a=df['Text'].values.tolist()
print(len(text_a))

24496


In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words= stopwords.words('arabic')
for index,text_ in enumerate(text_a):
    text_=" ".join(i for i in text_.split() if i not in stop_words) #Removing stopword
    text_a[index]=text_

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
#Stemming the word
stemmer = FarasaStemmer(interactive=True)
for index,text_ in enumerate(text_a):
    text_=" ".join(stemmer.stem(i) for i in text_.split())   
    text_a[index]=text_

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'farasa-api.qcri.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


100%|██████████| 241M/241M [00:26<00:00, 8.96MiB/s]


[2023-06-07 12:30:26,934 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
df['Text']=text_a[:len(df)]
df.head()

In [ ]:
num_cores = 2
def df_parallelize_run(df, func, num_cores=2):
    df_split = np.array_split(df, num_cores)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
df = df_parallelize_run(df, clean)

In [ ]:
df= shuffle(df,random_state=0)
df.head()

# **N-Gram generation (Only one token)**


In [ ]:
x=df.iloc[:,0]
y=df.iloc[:,1]
X=x.to_dict()

X=[]
for d in range(len(x)):
    b=x[d]
    X.append(b) 

In [ ]:
count_vect=CountVectorizer(ngram_range=(1,1))
X_train_counts=count_vect.fit_transform(X)
# X_train_counts = np.array(X_train_counts)
df_bow_sklearn = pd.DataFrame(X_train_counts.toarray(),columns=count_vect.get_feature_names_out())
df_bow_sklearn.head()

In [ ]:
df_bow_sklearn ['Label']= df['Label']

In [ ]:
df_bow_sklearn = shuffle(df_bow_sklearn)

In [ ]:
df_bow_sklearn['Label'] = df_bow_sklearn['Label'].replace([0, 1], ['Offensive', 'Benign'])

In [ ]:
# SSS=SSS.reset_index()
def list_token(doc , ack):
    temp1=pd.DataFrame(doc).transpose()
    temp2 = temp1.loc[:, (temp1 != 0).any(axis=0)]
    freq = temp2.drop(['Label'], axis=1)
    list_words =freq.columns.tolist()
    return list_words

In [ ]:
word_cl_dic={}
n = len(df_bow_sklearn)

In [ ]:
for i in range(0,len(df_bow_sklearn)):
    dt=df_bow_sklearn.iloc[i,]
    lst_words=list_token(dt,0)
    cl=dt['Label']
    for w in set(lst_words):
        word_cl_dic[w]=word_cl_dic.get(w,{})
        word_cl_dic[w][cl]=word_cl_dic[w].get(cl,0)
        word_cl_dic[w][cl]+=1


In [ ]:
word_cl_dic

In [ ]:
df_bow_sklearn['ID'] = df_bow_sklearn.index

In [ ]:
word_features={}
k=(df_bow_sklearn.shape[1]-2)
num_doc_cl=df_bow_sklearn.groupby(['Label'])['ID'].count()
num_doc_cl

# **PMI Calculation**


In [ ]:
for w in word_cl_dic.keys():
    dic = word_cl_dic[w]
    n_kw=0
    for cl in dic.keys():
        n_kw+=dic[cl] ## number of documents where the word occurs
    for cl in dic.keys():
        word_features[cl]=word_features.get(cl,[])
        n_kw_cl=dic[cl]
        n_dev_cl=num_doc_cl.loc[cl]
        p_w=n_kw/n
        p_cl = n_dev_cl/n
        p_w_cl= n_kw_cl/n
        
        pmi=log((p_w_cl)/(p_w*p_cl))
        if len(word_features[cl])<k:
            word_features[cl].append((w,pmi))
            if len(word_features[cl])==k:
                word_features[cl].sort(key=operator.itemgetter(1),reverse=True)                
        else:
            if word_features[cl][k-1][1]<pmi:
                word_features[cl][k-1]= (w,pmi)
                word_features[cl].sort(key=operator.itemgetter(1),reverse=True)   

In [ ]:
Offensive_words, benign_words= [], []
for field, value in word_features.items():
    if field == 'Offensive':
        Offensive_words = value
    else:
        benign_words = value
    
print(Offensive_words)

In [ ]:
print(benign_words)

In [ ]:
def Convert(tup, di):
    di = dict(tup)
    return di
     
# Driver Code
dictionary = {}
benign_words_to_dictionary=Convert(benign_words, dictionary)
print (benign_words_to_dictionary)

In [ ]:
dictionary_1 = {}
Offensive_words_to_dictionary=Convert(Offensive_words, dictionary_1)
print (benign_words_to_dictionary)

# **SO Calculation**


In [ ]:
Cdict = collections.defaultdict(int)
for key, val in itertools.chain(benign_words_to_dictionary.items(), benign_words_to_dictionary.items()):
    Cdict[key] -= val
     
print(dict(Cdict))

# **Generated Lexicon**

In [ ]:
Finall_Lexicon= pd.DataFrame({"key": Cdict.keys(), "value": Cdict.values()})
Finall_Lexicon.to_csv (r'Lexicon.csv', index=False, header=True, encoding ='utf-8-sig')

In [ ]:
Lexicon.head(10)

In [5]:
# with open('Lexicon.csv', 'w', encoding='utf-8-sig') as file:
#     writer = csv.DictWriter(file, fieldnames=Cdict.keys())
#     writer.writeheader()
#     writer.writerows(Cdict.values())
#Lexicon.head(10)